In [1]:
import sys
import os
from glob import glob
import json
import pandas

from langchain.agents import Tool

import agents.enemy as ae

In [2]:
questions = [
    {
        "question": "Initiative",
        "should_roll": True,
    },
    {
        "question": "Roll for Initiative",
        "should_roll": True,
    },
    {
        "question": "What is your Initiative modifier?",
        "should_roll": False
    },
    {
        "history": [
            "Roll Initiative",
            "Initiative Order:\n- Player: 24\n- You: 6\n\nCurrent State:\n- Player: 7 HP\n - You: 12 HP"
        ],
        "question": "What would you like to do?",
        "should_roll": False
    },
    {
        "question": "What is your AC?",
        "should_roll": False
    },
    {
        "question": "What is your Armor Class?",
        "should_roll": False
    },
    {
        "question": "What are your current hit points?",
        "should_roll": False
    },
    {
        "history": [
            "Roll for initiative!",
        ],    
        "question": "The fighter swings a longsword at you. He rolled a 15. With his +4 attack roll, the total to hit is 15 + 4 = 19.",
        "should_roll": False
    },
    {
        "history": [
            "Roll for initiative!",
        ],    
        "question": "You take 11 points of slashing damage from the player's longsword. Please update your current hit points.",
        "should_roll": False
    },
    {
        "history": [
            "Roll for initiative!",
            "You take 11 points of slashing damage from the player's longsword. Please update your current hit points.",
        ],    
        "question": "What are your current hit points?",
        "should_roll": False
    },
]

In [3]:
def fixed_dice_roll(query):
    return 4 # valid for any dice roll!

injected_tools = [
    Tool(
        name="RollDice",
        func=fixed_dice_roll,
        description="call this to get the result of rolling dice.",
    )
]

In [4]:
test_cases = []
for enemy_file in glob("*.json", root_dir=sys.path[0]):
    with open(os.path.join(sys.path[0], enemy_file)) as fh:
        e = json.loads(fh.read())
        for q in questions:
            test_cases.append({"e": e, "q": q})

In [28]:
def apply(case):
    combatant = ae.Enemy(name=case["e"]["name"], stats=case["e"], identifier=case["e"]["name"], injected_tools=injected_tools)
    exc = None
    if case["q"].get("history", None):
        for m in case["q"]["history"]:
            try:
                combatant.talk(m)
            except Exception as e:
                exc = e

    if not exc:
        try:
            response = combatant.talk(case["q"]["question"])
        except Exception as e:
            exc = e
    
    
    return {**case["q"], "enemy": case["e"]["name"], "output": exc or response["output"]}

In [29]:
import concurrent.futures
results = []
with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
    for out in executor.map(apply, test_cases):
        results.append(out)

0---- []
Initiative


> Entering new AgentExecutor chain...
0---- []
Roll for Initiative


> Entering new AgentExecutor chain...
0---- []
What is your Initiative modifier?


> Entering new AgentExecutor chain...

Invoking: `RollDice` with `1d20`


40---- []
Roll Initiative


> Entering new AgentExecutor chain...
0---- []
What is your Armor Class?


> Entering new AgentExecutor chain...
0---- []
What is your AC?


> Entering new AgentExecutor chain...

Invoking: `RollDice` with `1d20`


4My Initiative modifier is -2.

> Finished chain.
0---- []
What are your current hit points?


> Entering new AgentExecutor chain...
0---- []
Roll for initiative!


> Entering new AgentExecutor chain...
0---- []
Roll for initiative!


> Entering new AgentExecutor chain...
My Armor Class is 8.

> Finished chain.
0---- []
Roll for initiative!


> Entering new AgentExecutor chain...
My AC is 8.

> Finished chain.

Invoking: `RollDice` with `1d20`


40---- []
Roll for Initiative


> Entering new AgentExecuto

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [30]:
df = pandas.DataFrame(results)

In [31]:
pandas.set_option('display.max_colwidth', None)
pandas.set_option('display.width', None)

df[df["question"].str.contains("current hit points")][["enemy", "question", "output"]]

,enemy,question,output
6,Zombie,What are your current hit points?,My current hit points are 22.
8,Zombie,You take 11 points of slashing damage from the player's longsword. Please update your current hit points.,My current hit points are 11.
9,Zombie,What are your current hit points?,My current hit points are 11.
16,Goblin,What are your current hit points?,My current hit points are 7.
18,Goblin,You take 11 points of slashing damage from the player's longsword. Please update your current hit points.,"My current hit points were 7, and I took 11 points of slashing damage. So my new hit points are 7 - 11 = -4."
19,Goblin,What are your current hit points?,My current hit points are -4.
26,Giant Scorpion,What are your current hit points?,My current hit points are 52.
28,Giant Scorpion,You take 11 points of slashing damage from the player's longsword. Please update your current hit points.,My current hit points are 41.
29,Giant Scorpion,What are your current hit points?,My current hit points are 41.
36,Thugs,What are your current hit points?,My current hit points are 32.


In [37]:
df[["enemy","output"]]

,enemy,output
0,Zombie,"I rolled a 4 for initiative. With my initiative modifier of -2, my total initiative is 2."
1,Zombie,"I rolled a 4. With an initiative modifier of -2, my initiative is 2."
2,Zombie,My Initiative modifier is -2.
3,Zombie,"I rolled a 4. With a +3 to hit from my Slam attack, my total to hit is 7."
4,Zombie,My AC is 8.
...,...,...
108,Swamp Goblin,"My current hit points were 16, and I took 11 points of slashing damage. So my updated hit points are 5."
109,Swamp Goblin,My current hit points are 5.
110,Giant Toad,"I rolled a 4 for initiative. With my initiative modifier of +0, my total initiative is 4."
111,Giant Toad,"I rolled a 4 for initiative. With my initiative modifier of +0, my initiative is 4."


In [35]:
df[df["enemy"], "output"]

KeyError: ('enemy', 'output')

In [38]:
res = []
for t in test_cases:
    apply(t)

0---- []
Initiative


> Entering new AgentExecutor chain...

Invoking: `RollDice` with `1d20`


4I rolled a 4 for initiative. With my initiative modifier of -2, my total initiative is 2.

> Finished chain.
0---- []
Roll for Initiative


> Entering new AgentExecutor chain...

Invoking: `RollDice` with `1d20`


4I rolled a 4. With my initiative modifier of -2, my initiative is 2.

> Finished chain.
0---- []
What is your Initiative modifier?


> Entering new AgentExecutor chain...
My Initiative modifier is -2.

> Finished chain.
0---- []
Roll Initiative


> Entering new AgentExecutor chain...

Invoking: `RollDice` with `1d20`


4I rolled a 4. With an initiative modifier of -2, my initiative is 2.

> Finished chain.
Initiative Order:
- Player: 24
- You: 6

Current State:
- Player: 7 HP
 - You: 12 HP


> Entering new AgentExecutor chain...
The player goes first with an initiative of 24. Then, I go with an initiative of 2.

It is now the player's turn.

> Finished chain.
What would you like 